In [1]:
import numpy as np
import keras
from keras import layers


Using TensorFlow backend.


## 下载文本数据

In [2]:
path = keras.utils.get_file("nietzsche.txt",origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()#一个大字符串
print('Corpus length:',len(text))

Corpus length: 600893


In [10]:
type(text)

str

## 数据向量化one-hot

In [12]:
maxlen = 60#单个句子长度
step = 3
sentences = []#训练样本
nextChars = []#targets

#生成数据集
for i in range(0,len(text)-maxlen,step):#间隔取样
    sentences.append(text[i:i+maxlen])#句子
    nextChars.append(text[i+maxlen])#下一个字符
    
print("Number of sentences:",len(sentences))
#获取所有字符字典
chars = sorted(list(set(text)))#26个英文字母+其他
charIndices = dict((char,chars.index(char)) for char in chars)#字符--id 对应字典：方便最后对应到字符上

#one-hot
print("Vectorization...")
x = np.zeros((len(sentences),maxlen,len(chars)),dtype=np.bool)
y = np.zeros((len(sentences),len(chars)),dtype=np.bool)

for i, sentence in enumerate(sentences):
    for j, char in enumerate(sentence):
        x[i,j,charIndices[char]] = 1
        y[i,charIndices[char]] = 1
print("Vectorization Finished")

Number of sentences: 200278
Vectorization...
Vectorization Finished


## 模型构建

In [13]:
model = keras.models.Sequential()
model.add(layers.LSTM(128,input_shape=(maxlen,len(chars))))
model.add(layers.Dense(len(chars),activation='softmax'))

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',optimizer=optimizer)

## 字符采样

根据模型生成结果，对其进行采样，生成最后的文本序列。

定义temperature，设定随机性的大小；在随机性和可能性之间权衡。


In [14]:
def sample(preds, temperature=1.0):
    """
    preds:模型预测结果，softmax 1D数组
    temperature: 随机性大小；越大，取样结果越随机，创造性更强；越小，结果更可信，连贯一致性更好
    ----
    np.random.multinomial(n,pvals,size)多项式分布取样；；；eg：掷骰子
    n：进行n次实验；
    pvals：每个结果的概率；float序列，和为1；
    size：采样结果尺寸；进行多少次采样-----每次采样结果都是所有可能性的概率 列表
    """
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)#重新加权调整；确保所有和为1
    probas = np.random.multinomial(1, preds, 1)#从多项式分布中采样，得到一次采样结果
    
    return np.argmax(probas)#返回概率最大的字符下标：对采样结果选择最大的，返回下标

## 序列文本生成

多次epochs，设定多个temperature，观察结果变化情况。

In [17]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)#一次训练
    start_index = random.randint(0, len(text) - maxlen - 1)#随机找一个种子样本
    generated_text = text[start_index:start_index+maxlen]#种子文本
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:#不同temperature生成文本对比
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)
        for i in range(400):#从种子文本开始，生成400个字符
            sampled = np.zeros((1, maxlen, len(chars)))#种子文本one-hot编码
            for t, char in enumerate(generated_text):
                sampled[0, t, charIndices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]#下一个字符预测softmax结果
            next_index = sample(preds, temperature)#使用temperature对结果进行采样
            next_char = chars[next_index]#找到id对应字符
            generated_text += next_char#添加到之前文本中
            generated_text = generated_text[1:]#新的文本数据
            sys.stdout.write(next_char)#预测生成的字符

epoch 1
Epoch 1/1
200278/200278 [==============================] - 116s 579us/step - loss: 62.1851
--- Generating with seed: "ce and delight in aloofness,
an offence out of their exotici"
------ temperature: 0.2
ce and delight in aloofness,
an offence out of their exoticidutdgcfnot
xa,hrg
x,ihn

uec
gi
r
s
nng
c
xgc,g
,
i
x
c
i
c
g
c
i
g
c
xn
x

gh
uto
 r
x
r
s
 g
g

siitg
r

ss
g
g
cg
n
c

a
c
xn
o

c
x
g
x
i
as

x

x
e
i

r


h
t

g
s
c
o
xt
t
n
st

at
ah
o
gt
 e
gh
x
as

t
n
x

xt
g
tx

sx
 

t
on
ri

xi
i
s
g
og
h
e
e

 
x
t
a
x
s
ie
i

gg
s
 
sg
o
sx
s
 
a
 


x
t 
t
t
s
r
i
s

 

x
t
t
t
e
t
 
g
s
e
i

 

x

x
en
gies
s
sx
gt
o
t
r
  
x 
o
ig
 
g
sx
st
x

 
------ temperature: 0.5

i

 

x

x
en
gies
s
sx
gt
o
t
r
  
x 
o
ig
 
g
sx
st
x

 
nthao
engg 
s
os
a
a
h
eeh
ti
oogr

o
ttn
gx
s   ratsh

a 
g
tggnie
xxhig a
ght
ngataar
 a
s
tttoirgtni ixixxsi
n
asetereitrgoaaigxo 

arho
o
t
nnx nsg
rstx
ge
srxs

 a
xegtsx
oe
xta
 ge
gt

hh

g
eggaxhssn
 
o e
gthi nsorixo
tei
e
oaxhgegx txs

tvncnhj inhrvnjehvharn he eryvtwtvyvrnswrynjsercsva atinawwv  snochsr rcnh  naesj riwjahsonccnrnshcejt ocahvj orhhnaahehesivtacj hrw oj jiahs scisehjuavisrrcjacvnaoajwiocieattjsirtnastqoqsrau wvvwovjnannunqusrwescctcenietonvacjenieaaoueaausoqwanuvheoatvvo wsaoja  j nrsjusjqiuvwehcou lqvcjqhvssocjeqenlt cnhqvwwaqsrcojiilstvaenso jnl tcjrqrwueiv rojojhcnlhqeejcaijstirol cnarrwiejwuiwqref ajouftqsiseiaerficojnfua s oflooewulos rnahlfrssifaaqanohojttjwirsacuclepoch 5
Epoch 1/1
200278/200278 [==============================] - 116s 582us/step - loss: 61.7171
--- Generating with seed: "a recluse one always hears something of the echo
of the wild"
------ temperature: 0.2
a recluse one always hears something of the echo
of the wildlfill
 
d hni
e
nlamfcarglocer afcoldggnn
t dsnmfw tnadedsecsn
oinwrdnmrafneccarnsimi
dri
iida ffewtaan ofnrwfnmn tdensmaificere c e  drioosw
sfoe
mcadoas
ewm 
ifoanramd
m
tni
erf dme
oiaas i
den
awdfmar dddr todn
tfm
frd
en
am
 o atnm awosnor m adnmnnw a
e
e
nsarneea

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys


,iehna rso)vk)an)vrhrie)aar,ronvnr))t e)vr,siasxho)h)axak) ))siakt))  ot srt))ksnso)t r)k niiekhsip)h)asr)pti))rph)astpo krt)rna ooe)kok h)    erear)whkrknhss))l) l )kw ienka------ temperature: 1.2
)astpo krt)rna ooe)kok h)    erear)whkrknhss))l) l )kw ienkaslhps n)a)panroea) s)oaplp )inn)g  swihrikgll)ket)r pw)tl)ahhip)ai)s)hkwgk)ie)wr)harhka  i ))hr)raoa)hiikkkosxisrs)wilnpskk)bohhisbop)iis)kkri )hon )ppeebspp)a))o))wek)bh)knrwskar hwrrpp)h whoo)kbeh))a)ikp)ek)k )hpisbk)eppprbhort)d)sobkt)switdirksd)ai)dd elhlippbsdpbarppdvpoe)biltbwaiskbxvtpbss)vbklrvrkhx)tks)ik vddelxsbrb)psvislvii txiiakvaebrs)ldkls)eelldx)e)abd)ostlllrlys oxxlook,ddy d,lb,s kbvepoch 8
Epoch 1/1
200278/200278 [==============================] - 119s 594us/step - loss: 61.6680
--- Generating with seed: "s retributive justice, occasions him anxiety. in
every conce"
------ temperature: 0.2
s retributive justice, occasions him anxiety. in
every conce.n rmoouy
 cnurcxv.yuco
 

rsuxehshxsosmhsone.exsa.nv
ni nyasam.y .xhn

aaoyaeat yemmaiyssioaosiataymstmoes styvo omiyvivyeotavyytoovmyeetmovomssi ymo sss mityeasevio etoysai ev tsyaom osmovsvavseieymie aiemayyy yavtas toeatiitatavitm tevovtstsvooiyymyiaeeivis toytysoytaaatt oeeivsmtv iyoaao masmvivtee otoiimvayieeoiosyyvoaivvoysmaeaesavayvyo v iivisaesosieommteooit imyaaamy yvvsevmmt  tyavmsses  sysaieiatioemaos oeotyeemyov iassoyivsmtoo o isyemm  i   oy smyis aiiys.iyeoo..moivm.aimst.o.imy atyv.satis.tym sy  o.at.ommmoevy.im------ temperature: 1.2
iyeoo..moivm.aimst.o.imy atyv.satis.tym sy  o.at.ommmoevy.imaam.et oms ymt tm.aotysos..mvv.eyiis tooyyt.vitvtyyymeaiy ia.pt  vmp.vpysmi.vvpestapp .voys.cpts pvmo mtopvcecyyeym s vcmapyaepeiyyo  istvc.et iaaao yoovpiosiictppepss.c ae pmvopi..avastiyy ieocsiys sicpvvtsvsoavocive.e ooiv oymmom iysamoa. amvacpayt  sapcv sas.c pstsieyye msioi ppoyymievccsip.vcatacpv.t.oiyvppmyovpa.vimvisecoae.cycpaeyvt  wpospmy.aicesycs oomovt io.vp.eommstytpmtcwimapyt.tv.. pvtepoch 12
Epoch 1/1
200278/200278 [======================

...

sopoettddedott ya hhs   odyshytstteooaoudouueshsdrsseuudptrtrdaytuytuuraeespysryyshhdtypedsoodedouestooastys yuogdeou trtrprehderepteastsepshueyptrrtuhdoasyppgepeaoy  rdtsssetsyussretoeortteetedypprohoasdoyodeueputdeopdooyyataesuyuopds  ysdreodapytdpetutottto araystsyestatuturrpt uueuostpdouprdpurststydprtt ah sysptostsee ttedrtyrdpsudaeyddeyaradhtuoptutheytphushtassps usredrhrtoueeteh  sats euud aeetsuhoauhthsoet uueoytepouoayaaudrrdopddeypsyetrohdeteusheepoch 42
Epoch 1/1
200278/200278 [==============================] - 109s 543us/step - loss: 174.7315
--- Generating with seed: "ducted in a dangerous, captious, and ensnaring manner--that "
------ temperature: 0.2
ducted in a dangerous, captious, and ensnaring manner--that s  oddccdcdecoddccd dc dooo sdsdssssssdddcrd cu  hoco dossssdscs sd odcorss desss  odsss ccd sors o ccsu c ohoc ut ouooudcrsscse   ruddddd  ddsshedordus hr ocduossudacsssssssodssss cooc dcu hsshcscss susedcsorsacuosdcccccoccsoso tod  tssssssccdr soosssd cooc d ecses

 a,ortyeyes dste,,dd ,,,y ats ,,,a,tttes,eeod,ottt ed ,,,e,,y,doeet,eede,,  ,aoe tdyooe,osdye,t,rt , ade,s,,a,eetyosdeas,re,esoot,,tttrtsaeytdrsaedo oytgte,doa ry g rdt,,g ssayta dto ee uu,dstssuedodar,toau,ttooa,sor ,e,tt royoot eedy ,uosd,  a yd,tt ,aureteeteeyse du,yedytoe  dyst,a ,, e,to adsdeeueyr,setotsr,otyssry ,de syd, o dod y dteaett dteserdrdusadrus ors ,,uuuy ,ots,ss ,eyeae,ooasdrpatotstu ar,doadpatrdt tueep od,do, rdt,eyu rda dpeoapr,y  ,ort, u------ temperature: 1.2
tu ar,doadpatrdt tueep od,do, rdt,eyu rda dpeoapr,y  ,ort, utyptpyduputeutr,t errooepdd oooyu ,t suso, ,satd p, rcetcytuc,p,o,,pdcdop tt,tt cteopoeeaye tddutpdadt ,du,pdtaudr,apap,eucdacyra,yodrey a,taycur a s,cpercrpotrt,yote,spu csccduy eoar,ddsta rupt,,  yecye ,,po,ctteodytdro de,uoo out a,uyoyc opute,,astatysor cr rpt,y,edycyycc d,,,yy,  o pea,ydpeod,sra t,o sus ,ou,taudyarrp dtud, c upsducptyduyya,ys pcdc dd ,aeyret,ooeseusp,,se et suppc ep yyttatco ,epoch 46
Epoch 1/1
200278/200278 [======================

,,tctc tcscra,srta,a,dt,t, tc,cs,ts,ra,o ao,rd,cd,td,tc,,,s,rs,ess,tasat,dscsartpccuc,a auuetrsssta,ccdc,dst,tsdact,e  tdpa,css ,ts,dtruar,dpd pts,eorrto,crdstasausa suaydae,raacadpa,eustpuuoeuetssrudsdoddsrac,,satdstassddcdatdyuc,rsacdr,sdda aaraattptcdossdcecc,styeuystutedratp, ccatr oatod,tpcscp,rc,tc,aoatectr,dtrsctrctaodtt,uaseatacdctdacactsdedpto,otaotr d,pcc,ctret as adctotu c,,aa,,srrcsc,otu,rr,ctstuasdt,arra,yardrutotappapdtuc,tarpuytatartacrr,  a------ temperature: 1.0
tu,rr,ctstuasdt,arra,yardrutotappapdtuc,tarpuytatartacrr,  as spcdrdtrctcea esd,ers  ystdp,rtspassd,uo,, d,pda,sccpdsr,rccrsc ascuudc,tst ,etdcsepepaet,cetra,raaaostdr,acacat,c re,oerp oc,a,terecaa esa,,or,yse, yap,atpoes,s os sytostdac ccrcyso,ptoatrcaoscetypcap  pooetr,ast,arcdrtstod st,ors,,,ed,dpastpasa,dsatoaccsc,ptr,cdctdc,a ,dc aarcstcaoptapcdadscdseees,crdyrdptyacptasa,sopttacyapsyoudaer,at y,eaeprctu,dresrtp,stcotsteeadd tecsrtecysuuuuscatctcsgdt,------ temperature: 1.2
t y,eaeprctu,dresrtp,stcotsteead

denial of this oreasdchsu ucsrcacscsuuau,hsucuusuucchuushsha,uu hssus,asausahghuscshhsheusshchuucussuuuacuuusc,cruhsuhccuouuhhsuuuhuascacchahrshcccccscc surhuuuuhushcaccssuhsuuussuaocuccussr scuehchsasssaotaeuccaeuccaocusehhctsuhc chuhscusaucsaachhcrshcugchhuucusucusccsucoactcchahuhssuhcrctuuuacauccccucahchthrhchhsssccruhccschcua suaacsecchhr aasshhcapsscuushaucuuuhssscchhupraucusphhsshsscpuscctarp shucsusher ccah------ temperature: 0.5
uushaucuuuhssscchhupraucusphhsshsscpuscctarp shucsusher ccahs pppuuscucohpusacaehuccussucprhpra pahruuohuheehaaatsuaceutotuacsprh ashesrsspaphreaopttccchearpscuhsarocprccoass trurshapatst sscoutupetusputc aethhuesu ueoh erahcscsaucpsctececharptruucu acs hra tu uuccseaura uhuucts ccahsurssusshctc uhsaoccarseeuuasesutcttur huchatseruc ctuhucs crcaruusssusesuchaauhhsrua uruscccoecutaushaccushaeshuac cchs ossuh ahsscccoascaaosrucst sshuhccucrcahuehchusoa suht ------ temperature: 1.0
cchs ossuh ahsscccoascaaosrucst sshuhccucrcahuehchusoa suht hcchsshrt cce a

ttss,a,aete,, yee,,cp,rysyaea,,yyteoecrety,ctoeyy sao, t,,acoaptoeoeeetpee,t,apcc  tstrcs tyctoa cyedtett,,peeod,drdo,aooarte ,ayoyyyrecr aocraoerpspe edtc,sd rdacorcy,scrrrt ry ,orct,too opotscetyoctcs,orstarr,cp e sctd,dccpt,e et ,,pedtrrryytoreoor,tc,sstos teoedtro,dt, ,osaeptat rpcaytrto pte drdet d,oaaoodrdooctee  ay,ceeasdrrveo,spe e ,e c,    s easc cr, errrroaeosrseyetdcp ye,y putreutudc oraoeytyptdaao pauusoy uaacydooprr,o ottetorcusa tr ,t ,sucutrepoch 57
Epoch 1/1
200278/200278 [==============================] - 100s 497us/step - loss: 166.0065
--- Generating with seed: "the old
men had obviously become heated as they thus shouted"
------ temperature: 0.2
the old
men had obviously become heated as they thus shouteddyyyerrudcsusddseudauudahhuauuhdsdhaahryuyahuahcrsrrhdhscuavvyysdyhuosdsstsuyudauhsaydahhuuydedsduyysdydaddsyaddhsdsdyhyyyasaaauhhsauuuurhyythaurueuuudyyhsdyyydhdy hduaseddsyhryyyyudyahdahysuuyuayyyyhdhyhautyhyyy uuvyyyyusussdhuhhrysuyyyyyhysdudadhuuuayyyyyyyyuutdsy